In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import sklearn as skl
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/avanche-additional-features/Avalanche.csv
/kaggle/input/avalanche-snow-stability-dataset/snow_instability_field_data.csv
/kaggle/input/avalanche-snow-stability-dataset/read-me.pdf
/kaggle/input/avalanche-forecasting-prediction/StevensPass-GraceLakes_4790_feet_2019.csv
/kaggle/input/avalanche-accurate-features/Avalanche_accurate_featuers.csv
/kaggle/input/avalanche-forecasting-dataset/data_rf1_forecast.csv
/kaggle/input/avalanche-forecasting-dataset/dataset3_forecast.csv
/kaggle/input/avalanche-forecasting-dataset/read_me.xlsx
/kaggle/input/avalanche-forecasting-dataset/dataset3_nowcast.csv
/kaggle/input/avalanche-forecasting-dataset/dataset2.csv
/kaggle/input/avalanche-forecasting-dataset/data_rf2_tidy.csv
/kaggle/input/avalanche-forecasting-dataset/dataset1.csv


In [3]:
st_df = pd.read_csv('/kaggle/input/avalanche-snow-stability-dataset/snow_instability_field_data.csv',delimiter = ';')
ad_df = pd.read_csv('/kaggle/input/avanche-additional-features/Avalanche.csv',delimiter = ',')
ac_df = pd.read_csv('/kaggle/input/avalanche-accurate-features/Avalanche_accurate_featuers.csv',delimiter= ',')

# new_df = pd.DataFrame()
# new_df['Elevation '] = ad_df['Temperature']


st_df['Elevation (m a.s.l.)'].fillna(int(st_df['Elevation (m a.s.l.)'].mean()),inplace=True)
st_df['Avalanche_activity'].fillna(int(st_df['Avalanche_activity'].mean()),inplace=True)
st_df['RF_Regional _danger_level_forecast'].fillna(int(st_df['RF_Regional _danger_level_forecast'].mean()),inplace=True)
print(st_df.shape)
print(ad_df.shape)
print(ac_df.shape)

(599, 57)
(50, 19)
(204, 6)


In [4]:
ac_df['Prediction'].head()

0    2
1    0
2    0
3    1
4    0
Name: Prediction, dtype: int64

In [5]:


# Assuming df1 and df2 are your dataframes
df1 = pd.DataFrame({'danger_level': st_df['RF_Regional _danger_level_forecast'].astype(int),
                    'elevation': st_df['Elevation (m a.s.l.)'] ,'avalanche_act' : st_df['Avalanche_activity']})
print(df1['danger_level'].unique())
df2 = pd.DataFrame({'danger_level': ac_df['Prediction'].div(max(ac_df['Prediction'])+1).mul(5).round(0).astype(int),
                    'temperature': ac_df['Air Temperature'],'wind_speed': ac_df['Wind']})
ad_df['tilt_magnitude'] = np.sqrt(ad_df['tilt-x']**2 + ad_df['tilt-y']**2 + ad_df['tilt-z']**2)

df3 = pd.DataFrame({'danger_level': 6 - ad_df['tilt_magnitude'].div(max(ad_df['tilt_magnitude'])+1).mul(5).round(0).astype(int),'humidity': ad_df['Humidity']})

print(df2['danger_level'].unique())
print(df3['danger_level'].unique())
# Merge dataframes on the 'danger_level' k
merged_df = pd.merge(df1, df2, on='danger_level',how = 'inner') 
merged_df = pd.merge(merged_df,df3, on='danger_level',how='right')

# Display the merged dataframe
# df.to_csv(csv_file_path, index=False)
merged_df.head(100)
print(merged_df['danger_level'].unique())
print(merged_df.describe())
for i in merged_df.columns:
    merged_df[i].fillna(int(merged_df[i].mean()),inplace=True)
print(merged_df.describe())

[2 3 1 4]
[3 0 2]
[2 3 4 1]
[2 3 4 1]
        danger_level      elevation  avalanche_act    temperature  \
count  882249.000000  882244.000000  882244.000000  882244.000000   
mean        2.335440    2444.215818       0.206535      -8.673952   
std         0.472156     186.976027       0.404819      14.688612   
min         1.000000    1760.000000       0.000000     -34.000000   
25%         2.000000    2360.000000       0.000000     -22.000000   
50%         2.000000    2460.000000       0.000000      -8.000000   
75%         3.000000    2560.000000       0.000000       3.000000   
max         4.000000    2965.000000       1.000000      15.000000   

          wind_speed       humidity  
count  882244.000000  882249.000000  
mean       22.338598      58.677484  
std        11.495827       4.882538  
min         6.000000      48.000000  
25%        11.000000      55.000000  
50%        22.000000      59.000000  
75%        35.000000      63.000000  
max        40.000000      68.000000 

In [6]:
max(ad_df['tilt_magnitude'])

12.29186723000212

In [7]:
merged_df.columns

#snow depth , slope angle, snow thickness , temperature  , relative humidity

Index(['danger_level', 'elevation', 'avalanche_act', 'temperature',
       'wind_speed', 'humidity'],
      dtype='object')

In [8]:


X = merged_df[['elevation','temperature','wind_speed','humidity']]
y = merged_df['danger_level'].div(5).mul(100).astype(int)
# y = st_df[['RF_Regional _danger_level_forecast']].div(5)
y.tail()


882244    40
882245    40
882246    40
882247    40
882248    40
Name: danger_level, dtype: int64

In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# X = sc.fit_transform(X)

# from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder()
# y = ohe.fit_transform(y).toarray()

from sklearn.model_selection import train_test_split
train_x,X_test,train_y,y_test = train_test_split(X,y,test_size=0.1)

In [1]:
train_x.shape
train_y.head()
# X_test.head()

NameError: name 'train_x' is not defined

In [13]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=200)
# train_x.fillna(int(train_x.mean()),inplace=True)
history=model.fit(train_x, np.ravel(train_y,order='C'))


from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# train_x_scaled = scaler.fit_transform(train_x)

# test_x_scaled = scaler.transform(X_test)


In [15]:
model.score(X_test, y_test)

1.0

In [16]:

y_predicted = model.predict(X_test)
     

In [17]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_predicted)
cm

array([[    1,     0,     0],
       [    0, 58824,     0],
       [    0,     0, 29400]])

In [ ]:
merged_df.to_csv('mycsvfile.csv',index=False)

In [18]:
from sklearn.metrics import precision_score
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

y_val_pred=model.predict(X_test)

print(precision_score(y_test,y_val_pred,average=None))
     


print(recall_score(y_test,y_val_pred,average=None))
     


from sklearn.metrics import classification_report

print(classification_report(y_test, y_val_pred))

[1. 1. 1.]
[1. 1. 1.]
              precision    recall  f1-score   support

          20       1.00      1.00      1.00         1
          40       1.00      1.00      1.00     58824
          60       1.00      1.00      1.00     29400

    accuracy                           1.00     88225
   macro avg       1.00      1.00      1.00     88225
weighted avg       1.00      1.00      1.00     88225



In [ ]:
# from keras.models import Sequential
# from keras.layers import Dense, Dropout
# from keras.utils import to_categorical
# from sklearn.model_selection import train_test_split
# from keras.optimizers import Adam

# # Assuming train_x and train_y are your training data and labels
# # Split the data into training and testing sets
# train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=42)

# # Assuming train_y is a Pandas Series or NumPy array
# # Convert the target variable to categorical
# train_y_categorical = to_categorical(train_y)
# test_y_categorical = to_categorical(test_y)

# # create model
# model = Sequential()

# # add model layers
# model.add(Dense(50, activation='relu', input_dim=2))
# model.add(Dropout(0.2))
# model.add(Dense(25, activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(16, activation='relu'))
# model.add(Dropout(0.1))
# model.add(Dense(5, activation='softmax')) 

# custom_optimizer = Adam(learning_rate=0.002)

# # Compile the model with the custom optimizer
# model.compile(loss='categorical_crossentropy', optimizer=custom_optimizer, metrics=['accuracy'])
# print(train_x.shape)
# print(train_y_categorical.shape)

# # fit the model
# history = model.fit(train_x, train_y_categorical, epochs=100, batch_size=64, validation_split=0.2)

# # Evaluate the model on the test set



In [ ]:
# import matplotlib.pyplot as plt
# plt.plot(history.history['accuracy'])
# plt.ylim(0,0.6)


# plt.title('Model accuracy')
# plt.ylabel('Accuracy')
# plt.xlabel('Epoch')
# plt.legend(['Train'], loc='upper left')
# plt.show()

In [ ]:
# # y_pred = model.predict(X_test)
# # #Converting predictions to label
# # pred = list()
# # for i in range(len(y_pred)):
# #     pred.append(np.argmax(y_pred[i]))
# # #Converting one hot encoded test label to label
# # test = list()
# # for i in range(len(y_test)):
# #     test.append(np.argmax(y_test[i]))
# #     from sklearn.metrics import accuracy_score
# # a = accuracy_score(pred,test)
# # print('Accuracy is:', a*100)
# test_loss, test_accuracy = model.evaluate(test_x, test_y_categorical)
# print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [ ]:
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.ylim(0,1)
# plt.title('Model loss')
# plt.ylabel('Loss')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

In [ ]:
# new_data = np.array([[1.0, 2.0], [0.5, 1.5]])

# # Predict probabilities for each class
# predictions = model.predict(new_data)

# # Convert predicted probabilities to class labels
# predicted_labels = np.argmax(predictions, axis=1)

# # Display the results
# print("New Data:")
# print(new_data)
# print("Predicted Probabilities:")
# print(predictions)
# print("Predicted Labels:")
# print(predicted_labels/5)